In [1]:
import os,sys
import gdal
import subprocess as sub
import zipfile
import shutil
import rasterio

In [2]:
############## Unzip files ##############
path_to_zipped_files = r'C:\Users\Stavros\Desktop\Spetses'
os.chdir(path_to_zipped_files)

for zf in os.listdir(os.getcwd()):
    handle = zipfile.ZipFile(zf)
    handle.extractall('Unzipped')
handle.close()
for f in os.listdir('Unzipped'):
    with open("Unzipped\\file_names.txt", "a") as file:
        file.write(f+'\n')
############## Find the R, G, B and NIR files ##############
band_names = ['B02_10m','B03_10m','B04_10m','B08_10m']
new_fold_list = []
count = len(band_names) - 1
for root, dirs, files in os.walk('Unzipped'):
    for file in files:
        if file.endswith('.jp2'):
            filename, file_extension = os.path.splitext(file)
            if filename[-7:] in band_names:
                count = count + 1
                temp = 'image_'+str(count//len(band_names))
                new_folder = 'Unzipped\\' + temp
                if temp not in new_fold_list:
                    new_fold_list.append(temp)
                os.makedirs(new_folder,exist_ok=True)
                new_filename =  filename + file_extension
                shutil.move(os.path.join(root, file), new_folder+'\\'+ new_filename)  


In [3]:
############## Generate .vrt and final tiff images ##############
for folder in new_fold_list:
    path = path_to_zipped_files + '\\Unzipped\\' +folder
    os.chdir(path)
    files = [f for f in os.listdir(path) if f.endswith('.jp2')]
    new_name = files[0]
    new_name = new_name[7:-12]
    print('Processing: ' + new_name +'.tif')
    build_vrt = [r'C:\Program Files\QGIS 3.10\OSGeo4W.bat',
               'gdalbuildvrt', '-separate',new_name+'.vrt',
               '-te', 
                '708500','4125370','728670', '4139520', # clip extents

               files[0],files[1],files[2],files[3]
                ]
    create_tif = [r'C:\Program Files\QGIS 3.10\OSGeo4W.bat',
               'gdal_translate', new_name+'.vrt',new_name+'.tif']

    p1 = sub.Popen(build_vrt, stdout=sub.PIPE, stderr=sub.PIPE)
    stdout, stderr = p1.communicate()
    if p1.returncode != 0:
        print(stdout)
        print(stderr)

    p2 = sub.Popen(create_tif, stdout=sub.PIPE, stderr=sub.PIPE)
    stdout, stderr = p2.communicate()
    if p2.returncode != 0:
        print(stdout)
        print(stderr)